In [268]:
import numpy as np
import pandas as pd
import math
from sklearn.metrics import mean_squared_error
import random
from math import sqrt

In [269]:
df = pd.read_csv('C:/Users/HI/Documents/internship recommendation system/ml-25m/ratings.csv')
movies = pd.read_csv('C:/Users/HI/Documents/internship recommendation system/ml-25m/movies.csv')

In [270]:
new_df = pd.concat([movies.drop('genres', 1), movies['genres'].str.get_dummies(sep="|")], 1)

In [271]:
new_df.drop(new_df[new_df['(no genres listed)']==1].index,inplace=True)

In [272]:
new_df = new_df.drop(['(no genres listed)'], axis = 1) 

In [273]:
df = pd.merge(df, new_df, on='movieId')

In [274]:
def f(user):
    profile_1 = df[df['userId']==user]
    profile_1.drop(['movieId','title','timestamp','userId'], axis=1, inplace=True)
    profile_1["sum"] = profile_1.sum(axis=1)
    profile1_new = profile_1.loc[:,"rating":"Western"].div(profile_1["sum"], axis=0)
    return profile1_new

In [275]:
def euclidean(u1,u2,weights):
    s = pd.merge(df[df['userId']==u1], df[df['userId']==u2], how="inner", on="movieId")
    sum1 = 0
    similarity=0
    w1 = weights[0]
    w2 = weights[1]
    w3 = weights[2]
    w4 = weights[3]
    w5 = weights[4]
    w6 = weights[5]
    w7 = weights[6]
    w8 = weights[7]
    w9 = weights[8]
    w10 = weights[9]
    w11 = weights[10]
    w12 = weights[11]
    w13 = weights[12]
    w14 = weights[13]
    w15 = weights[14]
    w16 = weights[15]
    w17 = weights[16]
    w18 = weights[17]
    w19 = weights[18]
    w20 = weights[19]
    
    for index, row in s.iterrows():
           
        tempvar = (w1*((row.rating_x - row.rating_y)**2) + \
                w2*((row.Action_x - row.Action_y)**2) + \
                w3*((row.Adventure_x - row.Adventure_y)**2) + \
                w4*((row.Animation_x - row.Animation_y)**2) + \
                w5*((row.Children_x - row.Children_y)**2) + \
                w6*((row.Comedy_x - row.Comedy_y)**2) + \
                w7*((row.Crime_x - row.Crime_y)**2) + \
                w8*((row.Documentary_x - row.Documentary_y)**2) + \
                w9*((row.Drama_x - row.Drama_y)**2) + \
                w10*((row.Fantasy_x - row.Fantasy_y)**2) + \
                w11*((row.SciFi_x-row.SciFi_y)**2) + \
                w12*((row.Horror_x - row.Horror_y)**2) + \
                w13*((row.IMAX_x - row.IMAX_y)**2) + \
                w14*((row.Musical_x - row.Musical_y)**2) + \
                w15*((row.Mystery_x - row.Mystery_y)**2) + \
                w16*((row.Romance_x - row.Romance_y)**2) + \
                w17*((row.FilmNoir_x-row.FilmNoir_y)**2) + \
                w18*((row.Thriller_x - row.Thriller_y)**2) + \
                w19*((row.War_x - row.War_y)**2) + \
                w20*((row.Western_x - row.Western_y)**2)    
                  ) 
        tempvar = max(0, tempvar)
        tempvar = math.sqrt(tempvar)
        sum1 += tempvar
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
        
    return similarity

In [276]:
def similarUsers(user,user_weight_dict):
    k = 25
    sim_scores = []
    for i in df.userId.unique():
        s = euclidean(user, i, user_weight_dict)
        #s = correlation(user,i)
        #s = euclidean_similarity(user,i)
        sim_scores.append(s)
    simusers = {'users':df.userId.unique(), 'similarity_score':sim_scores}
    simusers = pd.DataFrame(simusers)
    simusers = simusers.nlargest(k, 'similarity_score')
    return simusers

In [277]:
def predictRating(user,movie,similar_user):
    n = 0
    predRating = 0
    k = 25
    #s = similarUsers(user)
    for i in similar_user['users']:
        rating = df[(df['userId']==i) & (df['movieId'] == movie)]['rating'].values
        sim = similar_user[similar_user['users']==i]['similarity_score'].values
        if(len(rating)!=0):
            predRating = predRating + (rating * sim)
    predRating = predRating/k 
    rating = df[df['userId']==user]['rating'].values
    average = np.average(rating)
    predRating = average+predRating
    if(predRating>5):
        predRating = 5
    return predRating

In [278]:
def getTopMovies1(user,similar_user):
    top_movies = []
    #s = similarUsers(testUser)
    for i in similar_user['users']:
        if i == user:
            continue
        movies = df[(df['userId']==i) & (df['rating']>4)]['movieId'].values
        for j in movies:
            if j not in top_movies:
                top_movies.append(j)
    return top_movies

In [279]:
def recommendation1(testUser,similar_user):
    mo = []
    p = []
    u = []
    m = getTopMovies1(testUser,similar_user)
    pr = 50
    if(len(m)<50):
        pr = len(m)
    for j in range(0,pr):
        predRating = predictRating(testUser,m[j],similar_user)
        mo.append(m[j])
        u.append(testUser)
        if np.isscalar(predRating):
            p.append(predRating)
        else:
            p.append(predRating[0])
    d = {'userId':u,'movieId':mo, 'predicted rating': p}
    df_u = pd.DataFrame(d)
    df_u = df_u.nlargest(50, 'predicted rating') 
    return df_u

In [280]:
def fitness_func(weights):
    w1 = weights[0]
    w2 = weights[1]
    w3 = weights[2]
    w4 = weights[3]
    w5 = weights[4]
    w6 = weights[5]
    w7 = weights[6]
    w8 = weights[7]
    w9 = weights[8]
    w10 = weights[9]
    w11 = weights[10]
    w12 = weights[11]
    w13 = weights[12]
    w14 = weights[13]
    w15 = weights[14]
    w16 = weights[15]
    w17 = weights[16]
    w18 = weights[17]
    w19 = weights[18]
    w20 = weights[19]
    
    w_sum = w1+w2+w3+w4+w5+w6+w7+w8+w9+w10+w11+w12+w13+w14+w15+w16+w17+w18+w19+w20
    
    w1 = w1/w_sum
    w2 = w2/w_sum
    w3 = w3/w_sum
    w4 = w4/w_sum
    w5 = w5/w_sum
    w6 = w6/w_sum
    w7 = w7/w_sum
    w8 = w8/w_sum
    w9 = w9/w_sum
    w10 = w10/w_sum
    w11 = w11/w_sum
    w12 = w12/w_sum
    w13 = w13/w_sum
    w14 = w14/w_sum
    w15 = w15/w_sum
    w16 = w16/w_sum
    w17 = w17/w_sum
    w18 = w18/w_sum
    w19 = w19/w_sum
    w20 = w20/w_sum
    
    w = [w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,w16,w17,w18,w19,w20]
    similar_user = similarUsers(testUser,w)
    df_u = recommendation1(testUser,similar_user)
    df_err = pd.merge(df,df_u,on=['movieId','userId'])
    if(df_err.shape[0]<1):
        return 100000
    rmse = sqrt(mean_squared_error(df_err['rating'], df_err['predicted rating']))
    return rmse

In [281]:
lb = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
ub = [255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255]

In [282]:
# implementation of PSO
# checking and defining weight inside the function; a random number between 0.5 and 1
def pso(fitness_func, lbound, ubound, swarm_size=2, max_iter=1, dimensions=20, c1=1.494, c2=1.494):
    x=[]            # positon of each particle
    x_pbest=[]      # best position of each particle 
    x_gbest=[]      # best global position of population
    v=[]            # velocity of each particle
    v_min=[]
    v_max=[]
    
    # assign initial random positions to the particles
    for i in range(swarm_size):
        x.append([])
        x_pbest.append([])
        for j in range(dimensions):
            x[i].append(random.uniform(lbound[j],ubound[j]))
            x_pbest[i].append(x[i][j])
            
    #calculate initial group best of the population
    for i in range(swarm_size):
        if i==0:
            x_gbest=x[i]
        elif fitness_func(x[i])<fitness_func(x_gbest):
            x_gbest=x[i]

    
    # calculate minimum and maximum boundaries of velocity vector
    for i in range(dimensions):
        v_min.append(-(ubound[i]-lbound[i])/100)
        v_max.append((ubound[i]-lbound[i])/100)
 
    # assign initial random velocities to the particles
    for i in range(swarm_size):
        v.append([])
        for j in range(dimensions):
            v[i].append(random.uniform(v_min[j],v_max[j]))
            
    for iter in range(max_iter):
        for i in range(swarm_size):
            for j in range(dimensions):
                r1=random.uniform(0,1)
                r2=random.uniform(0,1)
                weight = random.uniform(0.5, 1)
                # calculate new velocity for each particle
                v[i][j] = weight*(v[i][j]) + r1*c1*(x_pbest[i][j]-x[i][j]) + r2*c2*(x_gbest[j]-x[i][j])
                
                if v[i][j] > v_max[j]:
                    v[i][j] = v_max[j]
                
                if v[i][j] < v_min[j]:
                    v[i][j] = v_min[j]
                                 
                # calculate new position for each particle
                x[i][j] = x[i][j] + v[i][j]
                
                if x[i][j] > ubound[j]:
                    x[i][j] = ubound[j]
                    
                if x[i][j] < lbound[j]:
                    x[i][j] = lbound[j]
            
            if fitness_func(x[i]) < fitness_func(x_pbest[i]):
                x_pbest[i] = x[i]
                
            if fitness_func(x[i]) < fitness_func(x_gbest):
                x_gbest = x[i]
                
    return x_gbest,fitness_func(x_gbest)

In [283]:
def recommendation(testUser,similar_user):
    mo = []
    p = []
    u = []
    m = getTopMovies(testUser)
    pr = 50
    if(len(m)<50):
        pr = len(m)
    for j in range(0,pr):
        predRating = predictRating(testUser,m[j])
        mo.append(m[j])
        u.append(testUser)
        #print(j)
        if np.isscalar(predRating):
            p.append(predRating)
        else:
            p.append(predRating[0])
    d = {'userId':u,'movieId':mo, 'predicted rating': p}
    df_u = pd.DataFrame(d)
    df_u = df_u.nlargest(50, 'predicted rating') 
    return df_u

In [284]:
df_c = pd.DataFrame(columns=['userId','movieId','predicted rating'])
for testUser in df.userId.unique():
    xopt, fopt = pso(fitness_func, lb, ub)
    print('pso done')
    w = []
    for i in xopt:
        w.append(i/sum(xopt))
    similar_user = similarUsers(testUser,w)
    df_u = recommendation1(testUser,similar_user)
    df_c = pd.concat([df_u, df_c])
    print('recommendations for user', testUser, ':\n')
    l = 10
    if(len(df_u)<10):
        l = len(df_u['movieId'])
    for i in range(0,l):
        m = movies[movies['movieId']==df_u['movieId'][i]]['title'].values
        print(m[0])
    print('\n\n')

pso done
recommendations for user 1 :

Dark Knight, The (2008)
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Harry Potter and the Chamber of Secrets (2002)
Harry Potter and the Prisoner of Azkaban (2004)
Harry Potter and the Half-Blood Prince (2009)
Avatar (2009)
Ratatouille (2007)
WALL·E (2008)
Avengers, The (2012)
Deadpool (2016)



pso done
recommendations for user 5 :

Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
Mars Attacks! (1996)
Star Trek: First Contact (1996)
Dark Knight, The (2008)
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Harry Potter and the Chamber of Secrets (2002)
Harry Potter and the Prisoner of Azkaban (2004)
Harry Potter and the Half-Blood Prince (2009)
Avatar (2009)
Ratatouille (2007)



pso done
recommendations for user 7 :

Seven (a.k.a. Se7en) (1995)
Silence of the Lambs, The (1991)
Dumbo (1941)
Fight Club (1999)
Memento (2000)
28 Days Later (2002)
Evil Dead II (Dea

pso done
recommendations for user 73 :

Heat (1995)
Braveheart (1995)
Rob Roy (1995)
Forrest Gump (1994)
Fugitive, The (1993)
Platoon (1986)
Princess Bride, The (1987)
Back to the Future (1985)
Last of the Mohicans, The (1992)
Saving Private Ryan (1998)



pso done
recommendations for user 76 :

Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
Birdcage, The (1996)
Star Trek: First Contact (1996)
Lock, Stock & Two Smoking Barrels (1998)
Departed, The (2006)
Snatch (2000)
Boondock Saints, The (2000)
Layer Cake (2004)
Lucky Number Slevin (2006)
Crash (2004)



pso done
recommendations for user 78 :

Requiem for a Dream (2000)
Singin' in the Rain (1952)
Traffic (2000)
Memento (2000)
Beautiful Mind, A (2001)
Life Less Ordinary, A (1997)
Psycho (1960)
Rushmore (1998)
American Beauty (1999)
Kill Bill: Vol. 1 (2003)



pso done
recommendations for user 82 :

Lock, Stock & Two Smoking Barrels (1998)
Departed, The (2006)
Snatch (2000)
Boondock Saints, The (2000)
Layer Cake (2004)
Lucky Number Slevin (2

pso done
recommendations for user 153 :

Fargo (1996)
Dead Man Walking (1995)
Remains of the Day, The (1993)
Mary Shelley's Frankenstein (Frankenstein) (1994)
Madness of King George, The (1994)
Persuasion (1995)
Burnt by the Sun (Utomlyonnye solntsem) (1994)
Lock, Stock & Two Smoking Barrels (1998)
Departed, The (2006)
Snatch (2000)



pso done
recommendations for user 155 :

Usual Suspects, The (1995)
Braveheart (1995)
Stargate (1994)
Clear and Present Danger (1994)
Forrest Gump (1994)
Jungle Book, The (1994)
Fugitive, The (1993)
Jurassic Park (1993)
Dances with Wolves (1990)
Mission: Impossible (1996)



pso done
recommendations for user 156 :

Dark Knight, The (2008)
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Harry Potter and the Chamber of Secrets (2002)
Harry Potter and the Prisoner of Azkaban (2004)
Harry Potter and the Half-Blood Prince (2009)
Avatar (2009)
Ratatouille (2007)
WALL·E (2008)
Avengers, The (2012)
Deadpool (2016)



pso done
recommendations for user 219 :

Usual Suspects, The (1995)
Star Wars: Episode IV - A New Hope (1977)
Forrest Gump (1994)
Schindler's List (1993)
Saving Private Ryan (1998)
Fight Club (1999)
Green Mile, The (1999)
Shawshank Redemption, The (1994)
Departed, The (2006)
Dark Knight, The (2008)



pso done
recommendations for user 220 :

Easy Rider (1969)
Matilda (1996)
300 (2007)
10 Things I Hate About You (1999)
Muppet Treasure Island (1996)
Peter Pan (1953)
40-Year-Old Virgin, The (2005)
Goal! The Dream Begins (Goal!) (2005)
Chasing Liberty (2004)
Center Stage (2000)



pso done
recommendations for user 223 :

This Is Spinal Tap (1984)
Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)
Star Trek: First Contact (1996)
American History X (1998)
Fight Club (1999)
Green Mile, The (1999)
Dark Knight, The (2008)
Shutter Island (2010)
Inception (2010)
Interstellar (2014)



pso done
recommendations for user 226 :

Schindler's List (1993)
Dark Knight, The (2008)
Inception (2010)
Bo

pso done
recommendations for user 283 :

Twister (1996)
Independence Day (a.k.a. ID4) (1996)
That Thing You Do! (1996)
Mr. Holland's Opus (1995)
William Shakespeare's Romeo + Juliet (1996)
Star Trek: First Contact (1996)
Long Kiss Goodnight, The (1996)
Children of the Corn IV: The Gathering (1996)
French Twist (Gazon maudit) (1995)
Anne Frank Remembered (1995)



pso done
recommendations for user 288 :

Forrest Gump (1994)
Schindler's List (1993)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Matrix, The (1999)
Fight Club (1999)
Shawshank Redemption, The (1994)
Inception (2010)
Dark Knight Rises, The (2012)
Interstellar (2014)
Ex Machina (2015)



pso done
recommendations for user 290 :

Dark Knight, The (2008)
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Harry Potter and the Chamber of Secrets (2002)
Harry Potter and the Prisoner of Azkaban (2004)
Harry Potter and the Half-Blood Prince (2009)
Avatar (200

pso done
recommendations for user 364 :

Toy Story (1995)
Usual Suspects, The (1995)
Office Space (1999)
Shutter Island (2010)
True Lies (1994)
Spectre (2015)
Miss Congeniality (2000)
You Don't Mess with the Zohan (2008)
Grown Ups (2010)
Blended (2014)



pso done
recommendations for user 367 :

Toy Story (1995)
Fargo (1996)
James and the Giant Peach (1996)
Postman, The (Postino, Il) (1994)
Lone Star (1996)
Dead Man Walking (1995)
Sense and Sensibility (1995)
Leaving Las Vegas (1995)
Emma (1996)
Heat (1995)



pso done
recommendations for user 372 :

Usual Suspects, The (1995)
Grumpy Old Men (1993)
Shawshank Redemption, The (1994)
Departed, The (2006)
Dark Knight, The (2008)
Dark Knight Rises, The (2012)
Amazing Spider-Man, The (2012)
Godfather, The (1972)
One Flew Over the Cuckoo's Nest (1975)
Godfather: Part II, The (1974)



pso done
recommendations for user 373 :

Usual Suspects, The (1995)
Schindler's List (1993)
Thin Red Line, The (1998)
Shawshank Redemption, The (1994)
Simpsons 

pso done
recommendations for user 456 :

Toy Story (1995)
Usual Suspects, The (1995)
Independence Day (a.k.a. ID4) (1996)
Star Wars: Episode IV - A New Hope (1977)
Forrest Gump (1994)
Mrs. Doubtfire (1993)
Silence of the Lambs, The (1991)
Wizard of Oz, The (1939)
Citizen Kane (1941)
E.T. the Extra-Terrestrial (1982)



pso done
recommendations for user 460 :

Citizen Kane (1941)
Willy Wonka & the Chocolate Factory (1971)
Shawshank Redemption, The (1994)
Rear Window (1954)
Strictly Ballroom (1992)
My Life in Pink (Ma vie en rose) (1997)
Gods Must Be Crazy, The (1980)
Strangers on a Train (1951)
Nights of Cabiria (Notti di Cabiria, Le) (1957)
Planes, Trains & Automobiles (1987)



pso done
recommendations for user 462 :

Lock, Stock & Two Smoking Barrels (1998)
Departed, The (2006)
Snatch (2000)
Boondock Saints, The (2000)
Layer Cake (2004)
Lucky Number Slevin (2006)
Crash (2004)
13th Warrior, The (1999)
Payback (1999)
10th & Wolf (2006)



pso done
recommendations for user 468 :

Dark K

pso done
recommendations for user 529 :

Braveheart (1995)
Shawshank Redemption, The (1994)
Disclosure (1994)
Outbreak (1995)
Dances with Wolves (1990)
Silence of the Lambs, The (1991)
Beauty and the Beast (1991)
Star Wars: Episode IV - A New Hope (1977)
Jurassic Park (1993)
Abyss, The (1989)



pso done
recommendations for user 533 :

Toy Story (1995)
Desperado (1995)
Clerks (1994)
Pulp Fiction (1994)
Tombstone (1993)
Shawshank Redemption, The (1994)
Like Water for Chocolate (Como agua para chocolate) (1992)
Babe (1995)
Apollo 13 (1995)
Quiz Show (1994)



pso done
recommendations for user 534 :

Usual Suspects, The (1995)
Dances with Wolves (1990)
Get Shorty (1995)
Apollo 13 (1995)
Little Women (1994)
Indian in the Cupboard, The (1995)
Mr. Holland's Opus (1995)
Net, The (1995)
While You Were Sleeping (1995)
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)



pso done
recommendations for user 541 :

Office Space (1999)
Star Wars: Episode I - The Phantom Menace (1999)
Dinosaur (2000)
Armour o

KeyboardInterrupt: 

In [ ]:
for testUser in range(1,611):
    xopt, fopt = pso(fitness_func, lb, ub, swarm_size=1,max_iter=1)
    w = []
    for i in xopt:
        w.append(i/sum(xopt))
    similar_user = similarUsers(testUser,w)
    df_u = recommendation(testUser,similar_user)
    print('recommendations for user', testUser, ':\n')
    l = 10
    if(len(df_u)<10):
        l = len(df_u['movieId'])
    for i in range(0,l):
        m = movies[movies['movieId']==df_u['movieId'][i]]['title'].values
        print(m[0])
    print('\n\n')

In [285]:
df_err = pd.merge(df,df_c,on=['movieId','userId'])
rmse = sqrt(mean_squared_error(df_err['rating'], df_err['predicted rating']))
mse = mean_squared_error(df_err['rating'], df_err['predicted rating'])
print(mse)
print(rmse)

0.35489054150336746
0.5957269017791352
